# Verify Installation

In [1]:
import tokentoss
print(f"tokentoss version: {tokentoss.__version__}")
print(f"Config path: {tokentoss.get_config_path()}")

tokentoss version: 0.1.0
Config path: /Users/nicholasgrundl/Library/Application Support/tokentoss/client_secrets.json


# Check for Existing Config

In [2]:
from pathlib import Path

config_path = tokentoss.get_config_path()
if config_path.exists():
    print(f"Config exists at: {config_path}")
    perms = oct(config_path.stat().st_mode & 0o777)
    print(f"Permissions: {perms}")
else:
    print("No config found — will use ConfigureWidget next")

Config exists at: /Users/nicholasgrundl/Library/Application Support/tokentoss/client_secrets.json
Permissions: 0o600


# ConfigureWidget

In [3]:
from tokentoss import ConfigureWidget

cw = ConfigureWidget()
display(cw)

# Verify Config Was Saved

In [3]:
import json, os

config_path = tokentoss.get_config_path()
assert config_path.exists(), "Config file not created!"

perms = oct(config_path.stat().st_mode & 0o777)
print(f"File: {config_path}")
print(f"Permissions: {perms}")
assert perms == "0o600", f"Expected 0o600, got {perms}"

with open(config_path) as f:
    data = json.load(f)

print(f"Has 'installed' key: {'installed' in data}")
print(f"Client ID ends with: ...{data['installed']['client_id'][-20:]}")
print("Config saved correctly!")

File: /Users/nicholasgrundl/Library/Application Support/tokentoss/client_secrets.json
Permissions: 0o600
Has 'installed' key: True
Client ID ends with: ...oogleusercontent.com
Config saved correctly!


# GoogleAuthWidget — Fresh Auth

In [1]:
import tokentoss
tokentoss.enable_debug() 

In [2]:
from tokentoss import GoogleAuthWidget

widget = GoogleAuthWidget()
display(widget)

[tokentoss.widget DEBUG] Callback server started on port 61894
[tokentoss.widget DEBUG] Started callback server on port 61894


# Verify Authentication State

In [4]:
print(f"is_authenticated: {widget.is_authenticated}")
print(f"user_email: {widget.user_email}")
print(f"status: {widget.status}")

# Check module-level credentials
print(f"\ntokentoss.CREDENTIALS is set: {tokentoss.CREDENTIALS is not None}")
if tokentoss.CREDENTIALS:
    print(f"Credential type: {type(tokentoss.CREDENTIALS).__name__}")
    print(f"Token valid: {tokentoss.CREDENTIALS.valid}")
    print(f"Expired: {tokentoss.CREDENTIALS.expired}")

is_authenticated: True
user_email: nicholasgrundl@gmail.com
status: Signed in as nicholasgrundl@gmail.com

tokentoss.CREDENTIALS is set: True
Credential type: Credentials
Token valid: True
Expired: False


# Inspect AuthManager & Tokens

In [5]:
am = widget.auth_manager

print(f"AuthManager.is_authenticated: {am.is_authenticated}")
print(f"AuthManager.user_email: {am.user_email}")
print(f"Has refresh token: {am.credentials.refresh_token is not None}")

# Check token data in storage
token_data = am.storage.load()
if token_data:
    print(f"\nStored token data:")
    print(f"  user_email: {token_data.user_email}")
    print(f"  scopes: {token_data.scopes}")
    print(f"  expiry: {token_data.expiry}")
    print(f"  is_expired: {token_data.is_expired}")
    print(f"  has id_token: {bool(token_data.id_token)}")
    print(f"  has refresh_token: {bool(token_data.refresh_token)}")

AuthManager.is_authenticated: True
AuthManager.user_email: nicholasgrundl@gmail.com
Has refresh token: True

Stored token data:
  user_email: nicholasgrundl@gmail.com
  scopes: ['openid', 'https://www.googleapis.com/auth/userinfo.profile', 'https://www.googleapis.com/auth/userinfo.email']
  expiry: 2026-01-29T23:28:14+00:00
  is_expired: False
  has id_token: True
  has refresh_token: True


# Check Token File on Disk

In [6]:
from tokentoss.storage import FileStorage

fs = FileStorage()
print(f"Token file: {fs.path}")
print(f"Exists: {fs.path.exists()}")
if fs.path.exists():
    perms = oct(fs.path.stat().st_mode & 0o777)
    print(f"Permissions: {perms}")

Token file: /Users/nicholasgrundl/Library/Application Support/tokentoss/tokens.json
Exists: True
Permissions: 0o600


# Sign Out

In [7]:
widget.sign_out()

print(f"is_authenticated: {widget.is_authenticated}")
print(f"user_email: '{widget.user_email}'")
print(f"status: {widget.status}")
print(f"tokentoss.CREDENTIALS: {tokentoss.CREDENTIALS}")

is_authenticated: False
user_email: ''
status: Click to sign in
tokentoss.CREDENTIALS: None


# Re-display Widget After Sign Out

In [8]:
display(widget)

# Verify Re-Auth Worked

In [9]:
print(f"is_authenticated: {widget.is_authenticated}")
print(f"user_email: {widget.user_email}")
print(f"tokentoss.CREDENTIALS is set: {tokentoss.CREDENTIALS is not None}")

is_authenticated: True
user_email: nicholasgrundl@gmail.com
tokentoss.CREDENTIALS is set: True
